### Why Subscription Metadata Fails to Predict Churn: A Product Analytics Investigation

##### Can Subscription-level commercial metadata predict churn effectively?

##### Import Packages

In [75]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


##### Load dataset and clean

In [ ]:
# load the datasets
df_subscriptions = pd.read_csv("C:\\Users\\hanna\\Ayesha_Projects\\projects\\SaaS-churn-data\\data\\raw\\ravenstack_subscriptions.csv",
                 parse_dates=["start_date", "end_date"])


In [ ]:
# fill active subscriptions with today's date for end_date
active_end = df_subscriptions["end_date"].fillna(pd.Timestamp.today().normalize())

df_subscriptions["tenure_days"] = (active_end - df_subscriptions["start_date"]).dt.days

# print(df_subscriptions[["start_date", "end_date", "tenure_days"]].head())

##### Exploratory Lifecycle Analysis

##### Lifecycle and Subscriptions

In [ ]:
# create a new column for lifestage
def lifestage(row):
    if row["churn_flag"]: 
        return "churned"
    elif row["is_trial"]: 
        return "trial"
    else: 
        return "active"

df_subscriptions["lifestage"] = df_subscriptions.apply(lifestage, axis=1)
df_subscriptions["lifestage"].value_counts()

In [ ]:
# Count subscriptions per lifecycle stage
subscriptions_per_stage_counts = (
    df_subscriptions["lifestage"]
    .value_counts()
    .reindex(["trial", "active", "churned"])
)
subscriptions_per_stage_counts

In [ ]:
# Calculate percentage of total subscriptions for each lifecycle stage
subscriptions_per_stage_counts_percentages = subscriptions_per_stage_counts / subscriptions_per_stage_counts.sum()

subscriptions_per_stage_counts_percentages

In [ ]:
# count lifestages by plan tier 
subscriptions_per_stage_and_plan = (
    df_subscriptions
    .groupby(["plan_tier", "lifestage"], sort=False)
    .size()
    .rename("subscription_count")
    .reset_index()
)
subscriptions_per_stage_and_plan

In [ ]:
# convert to percentages within each plan tier
subscriptions_per_stage_and_plan["pct"] = (
    subscriptions_per_stage_and_plan["subscription_count"] / 
    subscriptions_per_stage_and_plan.groupby("plan_tier")["subscription_count"].transform("sum") * 100
).round(1)
subscriptions_per_stage_and_plan

In [ ]:
# calculate tenure statistics by lifestage
df_subscriptions.groupby("lifestage")["tenure_days"].describe()

In [ ]:
# visualize tenure by lifecycle stage
df_subscriptions.boxplot(column="tenure_days", by="lifestage", grid = False)
plt.suptitle("")
plt.title("Tenure by Lifecycle Stage")
plt.ylabel("Tenure (days)")
plt.show()

##### Churn timing and Subscriptions

In [ ]:
# Define churn timing categories based on tenure at churn
def churn_timing(row):
    if row["churn_flag"]:
        if row["tenure_days"] <= 7:
            return "early_churn"
        elif row["tenure_days"] <= 30:
            return "mid_churn"
        else:
            return "late_churn"
    else:
        return "not_churned"
    
df_subscriptions["churn_timing"] = df_subscriptions.apply(churn_timing, axis=1)
# df_subscriptions["churn_timing"].value_counts()

In [ ]:
# Count churn timing by plan tier
churned_df = df_subscriptions[df_subscriptions["churn_timing"].isin(["early_churn", "mid_churn", "late_churn"])]

churn_counts = (
    churned_df.groupby(["plan_tier", "churn_timing"], sort=False)
    .size()
    .reset_index(name="count")
)

In [ ]:
# convert to percentages within each plan tier
churn_counts["pct"] = (
    churn_counts["count"] / 
    churn_counts.groupby("plan_tier")["count"].transform("sum") * 100
).round(1)
churn_counts

In [ ]:
# sort by plan tier and churn timing for better visualization
churn_counts.sort_values(by=["plan_tier", "churn_timing"], inplace=True)
churn_counts

In [ ]:
# visualize churn timing by plan tier
plot_df_churn_counts= churn_counts.copy()
sns.catplot(
    data = plot_df_churn_counts,
    x= 'plan_tier', y= 'pct', hue='churn_timing', kind='bar',
    height=5, aspect=1.6
)
plt.title("Churn Timing by Plan Tier")
plt.ylabel("Percentage of Churned Subscriptions")
plt.xlabel("Plan Tier")
plt.show()

##### Load second table

In [ ]:
# churn events table
df_churn_events = pd.read_csv("C:\\Users\\hanna\\Ayesha_Projects\\projects\\SaaS-churn-data\\data\\raw\\ravenstack_churn_events.csv")
df_churn_events["churn_date"] = pd.to_datetime(df_churn_events["churn_date"])

df_churn_events = (
    df_churn_events
    .sort_values("churn_date")
    .groupby("account_id", as_index=False)
    .first()
)
# df_churn_events.head()

In [ ]:
merged_df = pd.merge(df_subscriptions, df_churn_events, on="account_id", how="left")
# merged_df.head()

In [ ]:
# quick sanity checks
""" print(df_subscriptions.shape)
print(merged_df.shape)

print(df_churn_events["account_id"].is_unique)
print(merged_df["reason_code"].notna().sum())

print("subscription rows:", df_subscriptions.shape[0])
print("churn events rows:", df_churn_events.shape[0])
print("merged rows:", merged_df.shape[0])

print("unique accounts in churn events:", df_churn_events["account_id"].nunique())
print("rows in churn_events:", len(df_churn_events)) """

##### Exploratory Data Analysis Cont'd

In [ ]:
# filter to churned users with a known reason
churn_reason_counts = (
    merged_df[
        (merged_df["lifestage"] == "churned") & 
        (merged_df["reason_code"].notna())
    ]
    .groupby(["churn_timing", "reason_code"], sort=False)
    .size()
    .reset_index(name="count")
)
churn_reason_counts

In [ ]:
# normalise to percentages
churn_reason_counts["pct"] = (
    churn_reason_counts["count"] / 
    churn_reason_counts.groupby("churn_timing")["count"].transform("sum") * 100
).round(1)
churn_reason_counts

##### Hypothesis Testing

In [ ]:
# compare auto renew flag rates between early churners and others
merged_df["early_churner"] = merged_df["churn_timing"] == "early_churn"
merged_df["auto_renew_flag"] = merged_df["auto_renew_flag"].fillna(False)
early_churn_auto_renew = merged_df.groupby("early_churner")["auto_renew_flag"].mean()
early_churn_auto_renew

In [ ]:
# compare upgrade flag rates between early churners and others
merged_df["early_churner"] = merged_df["churn_timing"] == "early_churn"
merged_df["upgrade_flag"] = merged_df["upgrade_flag"].fillna(False)
early_churn_upgrade = merged_df.groupby("early_churner")["upgrade_flag"].mean()
early_churn_upgrade

In [ ]:
# compare downgrade flag rates between early churners and others
merged_df["early_churner"] = merged_df["churn_timing"] == "early_churn"
merged_df["downgrade_flag"] = merged_df["downgrade_flag"].fillna(False)
early_churn_downgrade = merged_df.groupby("early_churner")["downgrade_flag"].mean()
early_churn_downgrade

In [ ]:
# compare tenure days between early churners and others
early_churn_tenure_mean= merged_df.groupby("early_churner")["tenure_days"].mean()
early_churn_tenure_mean

In [ ]:
early_churn_tenure_median= merged_df.groupby("early_churner")["tenure_days"].median()
early_churn_tenure_median

In [ ]:
# box plot of early churner tenure days distribution
sns.boxplot(x="early_churner", y="tenure_days", data=merged_df)
plt.title("Tenure Days Distribution by Early Churner Status")           
plt.xlabel("Early Churner")
plt.ylabel("Tenure Days")
plt.show()

In [ ]:
early_churn_seats_mean = merged_df.groupby("early_churner")["seats"].mean()
early_churn_seats_mean

In [ ]:
early_churn_seats_median = merged_df.groupby("early_churner")["seats"].median()
early_churn_seats_median

In [ ]:
# box plot early churner seats distribution
sns.boxplot(x="early_churner", y="seats", data=merged_df)
plt.title("Seats Distribution by Early Churner Status") 
plt.xlabel("Early Churner")
plt.ylabel("Seats")
plt.show()

In [ ]:
early_churn_mrr_mean = merged_df.groupby("early_churner")["mrr_amount"].mean()
early_churn_mrr_mean

In [ ]:
early_churn_mrr_median = merged_df.groupby("early_churner")["mrr_amount"].median()
early_churn_mrr_median

In [ ]:
# box plot of early churners mrr distribution
sns.boxplot(x="early_churner", y="mrr_amount", data=merged_df)
plt.title("MRR Distribution by Early Churner Status")   
plt.xlabel("Early Churner")
plt.ylabel("MRR Amount")
plt.show()

###### Early churn is not strongly differentiated by observable subscription-level features. At the subscription level, observable commercial attributes did not strongly differentiate early churners. This suggests that behavioural engagement metrics, rather than plan or revenue attributes, are likely stronger predictors of early churn.

##### Exploratory Data Analysis Cont'd - Ultra Early Churners 

In [ ]:
# create a column called tenure buckets for early churners
merged_df["tenure_bucket"] = pd.cut(merged_df["tenure_days"], bins=[0, 7, 30, 90, np.inf], labels=["0-7 days", "8-30 days", "31-90 days", "90+ days"])
merged_df["tenure_bucket"].loc[merged_df["early_churner"] == True]

In [ ]:
# count churn events in each tenure bucket
early_churn_tenure_bucket_counts = (
    merged_df[merged_df["early_churner"] == True]
    .groupby("tenure_bucket")
    .size()
    .reset_index(name="count")
)
early_churn_tenure_bucket_counts

In [ ]:
# normalise to percentages
early_churn_tenure_bucket_counts["pct"] = (
    early_churn_tenure_bucket_counts["count"] / 
    early_churn_tenure_bucket_counts["count"].sum() * 100
).round(1)  
early_churn_tenure_bucket_counts

In [ ]:
# define ultra early churn as 0-7 days
merged_df["ultra_early_churner"] = merged_df["tenure_days"] <= 7

In [ ]:
# calculate percentage of early churners that are ultra early
ultra_early_churn_pct = (
    early_churn_tenure_bucket_counts.loc[early_churn_tenure_bucket_counts["tenure_bucket"] == "0-7 days", "count"].values[0] / 
    early_churn_tenure_bucket_counts["count"].sum() * 100
).round(1)

##### Load third table for futher analysis

In [ ]:
# load feature usage data and merge with main dataframe
df_feature_usage = pd.read_csv("C:\\Users\\hanna\\Ayesha_Projects\\projects\\SaaS-churn-data\\data\\raw\\ravenstack_feature_usage.csv")
df_feature_usage["usage_date"] = pd.to_datetime(df_feature_usage["usage_date"]) 

In [ ]:
# join feature usage data to main dataframe on subscription_id, keeping all rows from main dataframe
merged_df = merged_df.merge(df_feature_usage, on="subscription_id", how="left")
merged_df.head()

##### Exploratory Data Analysis Cont'd

In [ ]:
# filter to ultra early churners and calculate feature usage counts
early_churn_feature_usage = (
    merged_df[merged_df["ultra_early_churner"] == True]
    .groupby("feature_name")
    .size()
    .reset_index(name="usage_count")
)
early_churn_feature_usage

In [ ]:
# compare ultra early churners to others for binary features like auto_renew_flag, upgrade_flag, downgrade_flag
binary_features = ["auto_renew_flag", "upgrade_flag", "downgrade_flag"] 
for feature in binary_features:
    feature_usage = merged_df.groupby("ultra_early_churner")[feature].mean()
    print(f"{feature} usage by ultra early churners vs others:")
    print(feature_usage)
    print() 


In [ ]:
# plot binary feature usage by ultra early churner status
for feature in binary_features:
    sns.catplot(x="ultra_early_churner", y=feature, data=merged_df, kind="bar")
    plt.title(f"{feature} Usage by Ultra Early Churner Status")
    plt.xlabel("Ultra Early Churner")
    plt.ylabel(f"{feature} Usage Rate")
    plt.show()  

In [ ]:
# compare ultra early churners to others for numeric features like tenure_days, seats, mrr_amount
numeric_features = ["tenure_days", "seats", "mrr_amount"]
for feature in numeric_features:
    feature_stats = merged_df.groupby("ultra_early_churner")[feature].describe()
    print(f"Statistics for {feature} by ultra early churner status:")
    print(feature_stats)
    print()

In [ ]:
# plot mean and median numeric feature values by ultra early churner status
for feature in numeric_features:
    sns.catplot(x="ultra_early_churner", y=feature, data=merged_df, kind="bar", estimator=np.mean)
    plt.title(f"Mean {feature} by Ultra Early Churner Status")
    plt.xlabel("Ultra Early Churner")
    plt.ylabel(f"Mean {feature}")
    plt.show() 

    sns.catplot(x="ultra_early_churner", y=feature, data=merged_df, kind="bar", estimator=np.median)
    plt.title(f"Median {feature} by Ultra Early Churner Status")
    plt.xlabel("Ultra Early Churner")
    plt.ylabel(f"Median {feature}")
    plt.show()

###### This dataset illustrates the limitation of subscription-state data in predicting early churn. It highlights the need for event-level engagement metrics to identify aha moments.

##### Predictive Modeling

In [ ]:
# prepare feature matrix
allowed_features = [
    "seats",
    "mrr_amount",
    "is_trial",
    "upgrade_flag",
    "downgrade_flag",
    "auto_renew_flag",
    "plan_tier",
    "billing_frequency"
]

X = merged_df[allowed_features]
X = pd.get_dummies(X, columns=["plan_tier", "billing_frequency"], drop_first=True)

y = merged_df["churn_flag"]

In [65]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
y_test.mean()

np.float64(0.09486718593968445)

In [66]:
# churn rate in test vs  train sets
print("Churn rate in train set:", y_train.mean())       
print("Churn rate in test set:", y_test.mean())       

Churn rate in train set: 0.09492659542594627
Churn rate in test set: 0.09486718593968445


In [69]:
# fit a logistic regression model
model = LogisticRegression(max_iter=1000, class_weight="balanced")
model.fit(X_train, y_train)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [73]:
# predict on test set and evaluate
y_pred = model.predict(X_test) 
print(classification_report(y_test, y_pred)) 

              precision    recall  f1-score   support

       False       0.91      0.49      0.64      4532
        True       0.10      0.53      0.17       475

    accuracy                           0.50      5007
   macro avg       0.50      0.51      0.40      5007
weighted avg       0.83      0.50      0.59      5007



high recall - low precision

In [74]:
# Get probabilities for positive class
y_pred_proba = model.predict_proba(X_test)[:, 1]  
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_proba))

ROC AUC Score: 0.5101140428299344


###### A logistic regression model using subscription-level commercial features yielded an ROC-AUC of 0.51, indicating minimal predictive power. This suggests that churn in this dataset is likely driven by behavioral engagement factors rather than plan configuration or revenue attributes.

###### Subscription-level commercial metadata is insufficient to model churn accurately

###### Proposal: We should Track Login frequency, Time to first meaningful action, Feature adoption depth, Session duration, Trial engagement milestones, Activation events